In [ ]:
from pycalphad import Model, Database, Workspace, variables as v
from pycalphad.property_framework.metaproperties import IsolatedPhase

import numpy as np

import scipy.integrate as spi
import scipy.optimize as opt
from scipy import integrate as spi

from sympy import symbols, preorder_traversal
import sympy as sp

import plotly.graph_objects as go
import matplotlib.pyplot as plt

from symbolic_hulls_func_aggr import *

In [ ]:
def get_symbols(expr):
    ordered_symbols = []
    seen = set()

    for node in preorder_traversal(expr):
        if node.is_Symbol and node not in seen:
            ordered_symbols.append(node)
            seen.add(node)

    return ordered_symbols

In [ ]:
db = Database("../TDDatabaseFiles_temp/ALCUY-2011.TDB")
phases = list(db.phases.keys())
# print(phases)

model1 = Model(db, ['Cu','Al', 'Y'],'AL42CU68Y10')
model2 = Model(db, ['Cu','Al', 'Y'],'AL7CU2Y3')

energy1_expr = sp.sympify(model1.GM)
energy2_expr = sp.sympify(model2.GM)

energy1_symbols = get_symbols(energy1_expr)
energy2_symbols = get_symbols(energy2_expr)

# print(energy1_symbols)
# print(energy2_symbols)

# Evaluate the function at some temperature
temperature = 600
energy1_expr = energy1_expr.subs(v.T, temperature)
energy2_expr = energy2_expr.subs(v.T, temperature)

display(energy1_expr)
display(energy2_expr)

# Sub out Y for 1-Al-Cu
energy1_expr = energy1_expr.subs(energy1_symbols[3], 1-energy1_symbols[1]-energy1_symbols[2]).evalf()
energy2_expr = energy2_expr.subs(energy2_symbols[1], 1-energy2_symbols[2]-energy2_symbols[3]).evalf()

# Convert to X andd Y for readibliity
x, y = symbols('x y')
energy1_expr = energy1_expr.subs(energy1_symbols[1], x).subs(energy1_symbols[2], y)
energy2_expr = energy2_expr.subs(energy2_symbols[2], x).subs(energy2_symbols[3], y)

# display(energy1_expr)
# display(energy2_expr)

# Note that x corresponds to Cu and y corresponds to Al

In [ ]:
X = np.linspace(0, 1, 100) 
Y = np.linspace(0, 1, 100)

# Create meshgrid
X, Y = np.meshgrid(X, Y)
mask = (X + Y) <= 1

# Create a triangular mask for the ternary system
X, Y = np.where(mask, X, np.nan), np.where(mask, Y, np.nan)

In [ ]:
# Evaluate the function
energy1_values = sp.lambdify((x, y), energy1_expr, 'numpy')(X, Y)
energy2_values = sp.lambdify((x, y), energy2_expr, 'numpy')(X, Y)

fig = go.Figure()

fig.add_trace(
    go.Surface(
        x=X, y=Y, z=energy1_values,
        colorscale='Viridis',
        opacity=0.7,
        name='AL42CU68Y10',
        showscale=False
    )
)

fig.add_trace(
    go.Surface(
        x=X, y=Y, z=energy2_values,
        colorscale='Viridis',
        opacity=0.7,
        name='AL7CU2Y3',
        showscale=False
    )
)

fig.update_layout(
    title="Al-Cu-Y Gibbs Energy",
    scene=dict(
        aspectmode='cube',
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Gibbs Energy'
        ),
    autosize=False,
    width=800,
    height=700,
)

In [ ]:
def build_2d_polynomial_basis(x, y, n):
    """
    Returns a list of Sympy monomials
    """
    basis = [1]
    for i in range(1, n+1):
        basis.append(x**i)
    for i in range(1, n+1):
        basis.append(y**i)

    return basis

def double_integral(sym_expression):
    """
    Numerically compute the integral of sym_expression(x,y) 
    """
    # Convert the sympy expression to a numeric Python function f_num(x,y).
    f_num = sp.lambdify((x, y), sym_expression, 'numpy')
    
    def integrand(yval, xval):
        return f_num(xval, yval)
    
    val, err = spi.dblquad(
        integrand,
        0,   # x-lower
        1,   # x-upper
        lambda xval: 0,  # y-lower
        lambda xval: 1 - xval  # y-upper; ternaty system
    )
    return val

def fit_polynomial_2D_galerkin(f_expr, n):
    """
    Fits a 2D polynomial p(x,y) = sum_{i=0..n, j=0..n} alpha_{i,j} * x^i y^j
    to the given Sympy expression f_expr(x,y), via
    least-squares on [0,1]^2. Returns the symbolic polynomial approximation.
    """
    # Build basis with cross terms
    basis = build_2d_polynomial_basis(x, y, n)
    n_basis = len(basis)  # = (n+1)^2

    # Construct the Gram matrix M and vector b
    M = sp.zeros(n_basis, n_basis)
    b_vec = sp.zeros(n_basis, 1)

    for i in range(n_basis):
        for j in range(n_basis):
            M[i, j] = double_integral(basis[i]*basis[j])
        b_vec[i, 0] = double_integral(f_expr*basis[i])

    # Solve the linear system M alpha = b
    alpha = M.LUsolve(b_vec)

    # Build the final polynomial
    p_approx = sum(alpha[i,0] * basis[i] for i in range(n_basis))
    p_approx_simple = sp.simplify(p_approx)
    
    return p_approx_simple

In [ ]:
n = 2
energy1_p = fit_polynomial_2D_galerkin(energy1_expr, n)
energy2_p = fit_polynomial_2D_galerkin(energy2_expr, n)

display(energy1_p)
display(energy2_p)

In [ ]:
energy1_p_values = sp.lambdify((x,y), energy1_p, 'numpy')(X, Y)
energy2_p_values = sp.lambdify((x,y), energy2_p, 'numpy')(X, Y)

In [ ]:
# Plot with Plotly
fig = go.Figure()

# Surface for original piecewise function
fig.add_trace(
    go.Surface(
        x=X, y=Y, z=energy1_values,
        colorscale='Viridis',
        opacity=0.7,
        name='Energy Function',
        showscale=False
    )
)

# Surface for approximate polynomial
fig.add_trace(
    go.Surface(
        x=X, y=Y, z=energy1_p_values,
        colorscale='RdBu',
        opacity=0.7,
        name='Approx p(x,y) with cross terms',
        showscale=False
    )
)

# # Surface for original piecewise function
# fig.add_trace(
#     go.Surface(
#         x=X, y=Y, z=energy2_values,
#         colorscale='Viridis',
#         opacity=0.7,
#         name='Energy Function',
#         showscale=False
#     )
# )

# # Surface for approximate polynomial
# fig.add_trace(
#     go.Surface(
#         x=X, y=Y, z=energy2_p_values,
#         colorscale='RdBu',
#         opacity=0.7,
#         name='Approx p(x,y) with cross terms',
#         showscale=False
#     )
# )

fig.update_layout(
    title="Energy and Polynomial Approximation", 
    scene=dict(
        aspectmode='cube',
        xaxis_title='Cu',
        yaxis_title='Al',
        zaxis_title='G'
    ),
    width=800,
    height=700
)

fig.show()

In [ ]:
# Round off the polynomials
energy1_p = energy1_p.replace(lambda term: term.is_Number, lambda term: int(round(term, 0)))
energy2_p = energy2_p.replace(lambda term: term.is_Number, lambda term: int(round(term, 0)))

boundary1 = hpboundry_f2_to_f1(energy1_p, energy2_p)
boundary2 = hpboundry_f2_to_f1(energy2_p, energy1_p)

display(boundary1)
display(boundary2)

In [ ]:
def compute_boundary_points(boundary, mesh_density):
    '''
    Compute the boundary points for the given boundary equation
    '''
    # Generate the 1D mesh
    space = np.linspace(0, 1, mesh_density)
    
    # Solve boundary=0 for y (treating x as the independent variable)
    solutions_y = sp.solve(sp.Eq(boundary, 0), y)
    # Solve boundary=0 for x (treating y as the independent variable)
    solutions_x = sp.solve(sp.Eq(boundary, 0), x)
    
    # Convert each Sympy solution to a NumPy-callable function
    sol_y_lambdas = [sp.lambdify(x, sol, 'numpy') for sol in solutions_y]
    sol_x_lambdas = [sp.lambdify(y, sol, 'numpy') for sol in solutions_x]

    # We'll accumulate all x- and y-values in lists and concatenate once.
    x_vals = []
    y_vals = []
    
    # Evaluate y = f(x) for each solution
    for f_y in sol_y_lambdas:
        x_vals.append(space)
        y_vals.append(f_y(space))
    
    # Evaluate x = f(y) for each solution
    for f_x in sol_x_lambdas:
        x_vals.append(f_x(space))
        y_vals.append(space)
    
    # Combine into final arrays
    X = np.concatenate(x_vals)
    Y = np.concatenate(y_vals)
    
    # Apply the mask: only keep points where X + Y <= 1
    # Replace invalid points with np.nan (or you could filter them out)
    mask = ((X + Y) <= 1) & (X >= 0) & (Y >= 0)
    X = X[mask]
    Y = Y[mask]
    
    return X, Y

In [ ]:
Xline1, Yline1 = compute_boundary_points(boundary1, 100)
Xline2, Yline2 = compute_boundary_points(boundary2, 100)

In [ ]:
# Plot with Plotly
fig = go.Figure()

# Surface for approximate polynomial
fig.add_trace(
    go.Surface(
        x=X, y=Y, z=energy1_p_values,
        colorscale='RdBu',
        opacity=0.7,
        name='Approx p(x,y) with cross terms',
        showscale=False
    )
)

# Surface for approximate polynomial
fig.add_trace(
    go.Surface(
        x=X, y=Y, z=energy2_p_values,
        colorscale='RdBu',
        opacity=0.7,
        name='Approx p(x,y) with cross terms',
        showscale=False
    )
)

fig.add_trace(
    go.Scatter3d(
        x=Xline1, y=Yline1, z=sp.lambdify((x,y), energy1_p, 'numpy')(Xline1, Yline1),
        mode='lines',
        name='Boundary 1'
    )
)

fig.add_trace(
    go.Scatter3d(
        x=Xline2, y=Yline2, z=sp.lambdify((x,y), energy2_p, 'numpy')(Xline2, Yline2),
        mode='lines',
        name='Boundary 2'
    )
)

fig.update_layout(
    title="Energy and Polynomial Approximation", 
    scene=dict(
        aspectmode='cube',
        xaxis_title='Cu',
        yaxis_title='Al',
        zaxis_title='G'
    ),
    width=800,
    height=700
)

fig.show()

## Testing

In [ ]:
from pycalphad import Model, Database, Workspace, variables as v
from pycalphad.property_framework.metaproperties import IsolatedPhase

import numpy as np

import scipy.integrate as spi
import scipy.optimize as opt
from scipy import integrate as spi

from sympy import symbols, preorder_traversal
import sympy as sp

import plotly.graph_objects as go
import matplotlib.pyplot as plt

from itertools import combinations

from symbolic_hulls_func_aggr import *

In [ ]:
db = Database("../TDDatabaseFiles_temp/ALCUY-2011.TDB")
phases = ['CU6Y']    # Test phases
print(phases)

In [ ]:
# Teperature
temperature = 200

# Variable dictionary here
x, y = symbols('x y')
symbol_dict = {'CU': x, 'AL': y, 'Y': 1-x-y, 'VA': 1}

# Polynomial order
n = 2

In [ ]:
comps = [list(sublattice)[0].name for sublattice in db.phases[phases[0]].constituents]
model = Model(db, comps, phases[0])
print(comps)

In [ ]:
model.GM.free_symbols